In [1]:
import torch
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torch import nn
from PIL import Image
import os
from torchvision import datasets, transforms
from collections import defaultdict
import albumentations as A
import numpy as np

### Finding different shapes and channels of images in the dataset

In [2]:
folder_path = r'D:\Suchit\Breast-Cancer-Detection\Dataset_BUSI_with_GT'
image_shapes, mask_shapes = set(), set()
channels = set()
images, masks = [], []
for folder in os.listdir(folder_path):
    for image in os.listdir(os.path.join(folder_path, folder)):
        with Image.open(os.path.join(folder_path, folder, image)) as img:
            if 'mask' in image:
                mask_shapes.add(img.size)
                masks.append(os.path.join(folder_path, folder, image))
            else:
                image_shapes.add(img.size)
                images.append(os.path.join(folder_path, folder, image))
            channels.add(img.mode)
print(f'image shape length - {len(image_shapes)} and mask shape length - {len(mask_shapes)}')
print(f'number of channels - {len(channels)}')

image shape length - 639 and mask shape length - 639
number of channels - 3


### Finding the average height and width of the images to resize them

In [3]:
folder_path = r'D:\Suchit\Breast-Cancer-Detection\Dataset_BUSI_with_GT'
height, width, num_samples = 0.0, 0.0, 0.0
for folder in os.listdir(folder_path):
    for image in os.listdir(os.path.join(folder_path, folder)):
        with Image.open(os.path.join(folder_path, folder, image)) as img:
            if 'mask' not in image:
                size = img.size
                height += size[1]
                width += size[0]
                num_samples += 1
height /= num_samples
width /= num_samples
print(f'average height = {height}')
print(f'average width = {width}')

average height = 501.4525641025641
average width = 615.6794871794872


Height and width is too large. Taking 256 * 256

### Finding the mean and standard deviation of the input images

In [4]:
def calculate_grayscale_stats(folder_path, resize_shape=(256, 256)):
    """Calculate mean and std for grayscale images"""
    print("Calculating grayscale statistics...")
    
    # Simple transform for grayscale statistics calculation
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize(resize_shape),
        transforms.ToTensor()
    ])
    
    class GrayscaleImageData(Dataset):
        def __init__(self, images, transform):
            self.images = images
            self.transform = transform
        
        def __len__(self):
            return len(self.images)
        
        def __getitem__(self, index):
            img_path = self.images[index]
            image = Image.open(img_path)
            return self.transform(image)
    
    # Get image paths (exclude masks)
    images = []
    for folder in os.listdir(folder_path):
        for image in os.listdir(os.path.join(folder_path, folder)):
            if 'mask' not in image:
                images.append(os.path.join(folder_path, folder, image))
    
    print(f"Found {len(images)} images for statistics calculation")
    
    dataset = GrayscaleImageData(images, transform)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    
    mean, std, num_samples = 0.0, 0.0, 0.0
    
    for data in loader:
        batch_size = data.size(0)
        data = data.view(batch_size, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        num_samples += batch_size
    
    mean /= num_samples
    std /= num_samples
    
    return mean.item(), std.item()

# Step 2: Calculate grayscale statistics for your dataset
grayscale_mean, grayscale_std = calculate_grayscale_stats(folder_path)
print(f'Grayscale mean = {grayscale_mean:.4f}, std = {grayscale_std:.4f}')

Calculating grayscale statistics...
Found 780 images for statistics calculation
Grayscale mean = 0.3279, std = 0.1998


### Managing the dataset

In [5]:
class Data(Dataset):
    def __init__(self, folder_path, transforms):
        super().__init__()
        self.images, self.masks, self.category = [], defaultdict(list), []
        classes = {'benign': 0, 'malignant': 1, 'normal': 2}
        self.transforms = transforms

        for folder in os.listdir(folder_path):
            for image in os.listdir(os.path.join(folder_path, folder)):
                img_path = os.path.join(folder_path, folder, image)
                if 'mask' in image:
                    self.masks[image[: image.index('_mask')]].append(img_path)
                else:
                    self.images.append(img_path)
                    self.category.append(classes[folder])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = self.images[index]
        key = os.path.basename(image_path)[: os.path.basename(image_path).index('.png')]
        mask_paths = self.masks[key]

        # Load image and masks as numpy arrays
        image = np.array(Image.open(image_path).convert('L'))
        masks = [np.array(Image.open(x).convert('L')) for x in mask_paths]

        # Combine masks
        mask = np.sum(np.stack(masks), axis=0)
        mask = np.clip(mask, 0, 1)  # ensure binary

        # Apply albumentations
        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask, self.category[index]

from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=[grayscale_mean], std=[grayscale_std]),
    ToTensorV2()
])

dataset = Data(folder_path, transform)


### Model

In [6]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        self.down1 = DoubleConv(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.down2 = DoubleConv(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.down3 = DoubleConv(128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.down4 = DoubleConv(256, 512)
        self.pool4 = nn.MaxPool2d(2)

        self.bottleneck = DoubleConv(512, 1024)

        self.up4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv4 = DoubleConv(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv3 = DoubleConv(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv2 = DoubleConv(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv1 = DoubleConv(128, 64)

        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        d1 = self.down1(x)
        p1 = self.pool1(d1)
        d2 = self.down2(p1)
        p2 = self.pool2(d2)
        d3 = self.down3(p2)
        p3 = self.pool3(d3)
        d4 = self.down4(p3)
        p4 = self.pool4(d4)
        bn = self.bottleneck(p4)

        u4 = self.up4(bn)
        u4 = torch.cat([u4, d4], dim=1)
        u4 = self.conv4(u4)
        u3 = self.up3(u4)
        u3 = torch.cat([u3, d3], dim=1)
        u3 = self.conv3(u3)
        u2 = self.up2(u3)
        u2 = torch.cat([u2, d2], dim=1)
        u2 = self.conv2(u2)
        u1 = self.up1(u2)
        u1 = torch.cat([u1, d1], dim=1)
        u1 = self.conv1(u1)

        out = self.final_conv(u1)
        return out

class Predictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size= 3),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size= 3),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size= 3),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, kernel_size= 3),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )
    def forward(self, x):
        return self.conv(x)

class UNetPredictor(nn.Module):
    def __init__(self, unet, predictor):
        super().__init__()
        self.unet = unet
        for param in self.unet.parameters():
            param.requires_grad = False
        self.predictor = predictor

    def forward(self, x):
        seg_out = self.unet(x)
        pred_out = self.predictor(seg_out)
        return pred_out
    

unet = UNet()
unet = unet.to('cuda')
predictor = Predictor()
predictor = predictor.to('cuda')

### Weighted Random Sampling

In [7]:
targets = dataset.category
class_counts = np.bincount(targets)
class_weight = 1.0 / class_counts
sample_weight = [class_weight[label] for label in targets]

sampler = WeightedRandomSampler(
    weights= sample_weight,
    num_samples= len(sample_weight),
    replacement= True
)

### DataLoader, optimzer and loss function

In [ ]:
trian_loader = DataLoader(dataset, batch_size= 16, sampler= sampler)
test_loader = DataLoader(dataset, batch_size=16, sampler= sampler)

criterion_unet = nn.BCEWithLogitsLoss()
criterion_prediction = nn.CrossEntropyLoss()

optimizer_unet = torch.optim.AdamW(unet.parameters(), lr= 0.001, weight_decay= 0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_unet,
    mode='min',       # 'min' for loss, 'max' for accuracy/metric
    factor=0.1,       # multiply LR by this factor
    patience=10,       # wait for 5 epochs before reducing LR
)

In [ ]:
from tqdm import trange

train_loss_unet, test_loss_unet = [], []

num_epochs = 500
patience = 20
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in trange(num_epochs, desc="Epochs"):
    unet.train()
    running_loss = 0.0
    for images, masks, labels in trian_loader:
        images = images.to('cuda')
        masks = masks.to('cuda').view(-1, 1, 256, 256).float()
        # labels = labels.to('cuda')

        # --- UNet mask loss ---
        optimizer_unet.zero_grad()
        mask_pred = unet(images)
        loss_mask = criterion_unet(mask_pred, masks)
        loss_mask.backward()
        optimizer_unet.step()

        # --- UNetPredictor classification loss ---
        # optimizer_unet_predictor.zero_grad()
        # class_pred = model(images)
        # loss_class = criterion_prediction(class_pred, labels)
        # loss_class.backward()
        # optimizer_unet_predictor.step()

        running_loss += loss_mask.item()
        # running_loss += loss_mask.item() + loss_class.item()

    avg_loss = running_loss / len(trian_loader)

    # Validation
    unet.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, masks, labels in test_loader:
            images = images.to('cuda')
            masks = masks.to('cuda').view(-1, 1, 256, 256).float()
            # labels = labels.to('cuda')
            mask_pred = unet(images)
            # class_pred = model(images)
            loss_mask = criterion_unet(mask_pred, masks)
            # loss_class = criterion_prediction(class_pred, labels)
            val_loss += loss_mask.item()
            # val_loss += loss_mask.item() + loss_class.item()
            # _, predicted = torch.max(class_pred, 1)
            # correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_loss /= len(test_loader)

    scheduler.step(val_loss)

    # val_acc = 100 * correct / total

    # print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f}")
    train_loss_unet.append(avg_loss)
    test_loss_unet.append(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(unet.state_dict(), 'unet1.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

import pandas as pd
train_loss_unet = pd.DataFrame(train_loss_unet)
train_loss_unet.to_csv('train_loss_unet1.csv', index= False, header= False)
test_loss_unet = pd.DataFrame(test_loss_unet)
test_loss_unet.to_csv('test_loss_unet1.csv', index= False, header= False)

Epochs:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch [1/500] Train Loss: 0.2007 | Val Loss: 0.2183


Epochs:   0%|          | 1/500 [00:59<8:17:45, 59.85s/it]

Epoch [2/500] Train Loss: 0.1837 | Val Loss: 0.1767


Epochs:   0%|          | 2/500 [01:59<8:13:48, 59.50s/it]

...

Epoch [274/500] Train Loss: 0.0098 | Val Loss: 0.0086


Epochs:  55%|█████▌    | 275/500 [5:15:20<4:42:33, 75.35s/it]

Epoch [275/500] Train Loss: 0.0096 | Val Loss: 0.0084


Epochs:  55%|█████▌    | 275/500 [5:16:35<4:19:01, 69.07s/it]

Epoch [276/500] Train Loss: 0.0095 | Val Loss: 0.0085
Early stopping at epoch 276


### Initializing the entire model

In [15]:
model = UNetPredictor(unet= unet, predictor= predictor)
model = model.to('cuda')

### Defining the optimizer and scheduler

In [16]:
optimizer_unet_predictor = torch.optim.AdamW(model.parameters(), lr = 0.001, weight_decay= 0.001)

scheduler_predictor = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_unet,
    mode='min',       # 'min' for loss, 'max' for accuracy/metric
    factor=0.1,       # multiply LR by this factor
    patience=10,       # wait for 5 epochs before reducing LR
)

In [17]:
from tqdm import trange

train_loss_unetpredictor, test_loss_unetpredictor = [], []

num_epochs = 500
patience = 20
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in trange(num_epochs, desc="Epochs"):
    model.train()
    running_loss = 0.0
    for images, masks, labels in trian_loader:
        images = images.to('cuda')
        masks = masks.to('cuda').view(-1, 1, 256, 256).float()
        labels = labels.to('cuda')

        # --- UNet mask loss ---
        # optimizer_unet.zero_grad()
        # mask_pred = unet(images)
        # loss_mask = criterion_unet(mask_pred, masks)
        # loss_mask.backward()
        # optimizer_unet.step()

        # --- UNetPredictor classification loss ---
        optimizer_unet_predictor.zero_grad()
        class_pred = model(images)
        loss_class = criterion_prediction(class_pred, labels)
        loss_class.backward()
        optimizer_unet_predictor.step()

        # running_loss += loss_mask.item()
        running_loss += loss_class.item()

    avg_loss = running_loss / len(trian_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, masks, labels in test_loader:
            images = images.to('cuda')
            masks = masks.to('cuda').view(-1, 1, 256, 256).float()
            labels = labels.to('cuda')
            mask_pred = unet(images)
            class_pred = model(images)
            # loss_mask = criterion_unet(mask_pred, masks)
            loss_class = criterion_prediction(class_pred, labels)
            # val_loss += loss_mask.item()
            val_loss += loss_class.item()
            _, predicted = torch.max(class_pred, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_loss /= len(test_loader)

    scheduler_predictor.step(val_loss)

    val_acc = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    # print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f}")
    train_loss_unetpredictor.append(avg_loss)
    test_loss_unetpredictor.append(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'model1.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

import pandas as pd
train_loss_unetpredictor = pd.DataFrame(train_loss_unetpredictor)
train_loss_unetpredictor.to_csv('train_loss_predictor1.csv', index= False, header= False)
test_loss_unetpredictor = pd.DataFrame(test_loss_unetpredictor)
test_loss_unetpredictor.to_csv('test_loss_predictor1.csv', index= False, header= False)

Epochs:   0%|          | 0/500 [00:00<?, ?it/s]

Epoch [1/500] Train Loss: 18.3328 | Val Loss: 1.0652 | Val Acc: 70.90%


Epochs:   0%|          | 1/500 [01:18<10:49:38, 78.11s/it]

Epoch [2/500] Train Loss: 0.7551 | Val Loss: 0.6705 | Val Acc: 74.87%


Epochs:   0%|          | 2/500 [02:22<9:42:50, 70.22s/it] 

...

Epoch [29/500] Train Loss: 1.1006 | Val Loss: 1.0987 | Val Acc: 34.74%


Epochs:   6%|▌         | 30/500 [26:15<6:33:04, 50.18s/it]

Epoch [30/500] Train Loss: 1.0992 | Val Loss: 1.0988 | Val Acc: 34.23%


Epochs:   6%|▌         | 30/500 [27:06<7:04:34, 54.20s/it]

Epoch [31/500] Train Loss: 1.1006 | Val Loss: 1.0992 | Val Acc: 31.28%
Early stopping at epoch 31


In [ ]:
from tqdm import trange

for param in model.layer.parameters():
    param.requires_grad = True

train_loss_unetpredictor, test_loss_unetpredictor = [], []

num_epochs = 500
patience = 20
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in trange(num_epochs, desc="Epochs"):
    model.train()
    running_loss = 0.0
    for images, masks, labels in trian_loader:
        images = images.to('cuda')
        masks = masks.to('cuda').view(-1, 1, 256, 256).float()
        labels = labels.to('cuda')

        # --- UNetPredictor classification loss ---
        optimizer_unet_predictor.zero_grad()
        class_pred = model(images)
        loss_class = criterion_prediction(class_pred, labels)
        loss_class.backward()
        optimizer_unet_predictor.step()

        # running_loss += loss_mask.item()
        running_loss += loss_class.item()

    avg_loss = running_loss / len(trian_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, masks, labels in test_loader:
            images = images.to('cuda')
            masks = masks.to('cuda').view(-1, 1, 256, 256).float()
            labels = labels.to('cuda')
            mask_pred = unet(images)
            class_pred = model(images)
            # loss_mask = criterion_unet(mask_pred, masks)
            loss_class = criterion_prediction(class_pred, labels)
            # val_loss += loss_mask.item()
            val_loss += loss_class.item()
            _, predicted = torch.max(class_pred, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_loss /= len(test_loader)

    scheduler_predictor.step(val_loss)

    val_acc = 100 * correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    # print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {avg_loss:.4f} | Val Loss: {val_loss:.4f}")
    train_loss_unetpredictor.append(avg_loss)
    test_loss_unetpredictor.append(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'model1.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

import pandas as pd
train_loss_unetpredictor = pd.DataFrame(train_loss_unetpredictor)
train_loss_unetpredictor.to_csv('train_loss_predictor1.csv', index= False, header= False)
test_loss_unetpredictor = pd.DataFrame(test_loss_unetpredictor)
test_loss_unetpredictor.to_csv('test_loss_predictor1.csv', index= False, header= False)